## nnImport

In [30]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [31]:
device = (
    "cuda"
    if torch.cuda.is_available()

    else "mps"
    if torch.backends.mps.is_available() #apple m1/m2의 mps가 사용가능하면

    else "cpu"

#     if torch.cuda.is_available():
#         device = "cuda"
#     elif torch.backends.mps.is_available():
#         device = "mps"
#     else:
#         device = "cpu"
)
print(f"Using {device} device")

Using cuda device


In [32]:
class NeuralNetwork(nn.Module): #nnModule을 상속받아 pytorch 신경망모델생성
    def __init__(self): 
        super().__init__() #부모클래스 생성자 호출
        self.flatten = nn.Flatten() #입력이미지 1차원으로 평탄화 (Flatten()층은 2D이미지를 1D 벡터로 변환)
        self.linear_relu_stack = nn.Sequential( #nn.Sequential()은 여러층을 순차적으로 쌓을때 사용
            nn.Linear(28 * 28, 512),    # nn.Linear() 28 x 28크기의 이미지를 받아 512개의 출력 노드생성
            nn.ReLU(), # 활성화 함수로 신경망의 비선형성을 추가해서 복잡한 패턴을 학습할 수 있도록 돕는다. ReLU는 입력이 양수일 경우에는 그대로 출력하고, 음수일 경우에는 0을 출력
            nn.Linear(512, 512),# 512개입력을 받아 다시 512개의 출력으로 변환
            nn.ReLU(),
            nn.Linear(512, 10), # 512개의 입력을 받아 다시 10개의 출력으로 변환
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [33]:
x = torch.rand(1,28,28,device = device) #배치크기,이미지 높이, 이미지 너비 즉 3차원 텐서가 생성됨
x

tensor([[[0.0228, 0.6690, 0.8058, 0.4911, 0.6588, 0.6225, 0.9246, 0.5762,
          0.7925, 0.3222, 0.8062, 0.8526, 0.4853, 0.2271, 0.9539, 0.6439,
          0.0247, 0.0550, 0.8429, 0.7862, 0.6860, 0.2651, 0.1220, 0.2964,
          0.6204, 0.2916, 0.9505, 0.4246],
         [0.9108, 0.9258, 0.5849, 0.5882, 0.1736, 0.3016, 0.2808, 0.8464,
          0.0616, 0.3477, 0.1374, 0.3285, 0.6806, 0.9513, 0.5344, 0.8922,
          0.2697, 0.9956, 0.5833, 0.4988, 0.5098, 0.2670, 0.8972, 0.1031,
          0.6674, 0.5793, 0.0792, 0.7824],
         [0.5826, 0.7797, 0.5781, 0.2102, 0.8577, 0.7628, 0.7249, 0.9451,
          0.8688, 0.4513, 0.5268, 0.1834, 0.7481, 0.9836, 0.7784, 0.7895,
          0.0518, 0.6023, 0.8163, 0.5217, 0.5166, 0.3642, 0.5564, 0.8072,
          0.6522, 0.5593, 0.7845, 0.7567],
         [0.4936, 0.1202, 0.7715, 0.2122, 0.8008, 0.0243, 0.3622, 0.5854,
          0.0191, 0.6571, 0.6343, 0.9238, 0.4229, 0.2430, 0.1817, 0.1597,
          0.3539, 0.4099, 0.3871, 0.1171, 0.4104, 0.5855,

In [34]:
model = NeuralNetwork().to(device) #모델 인스턴스 생성
logits = model(x) #모델에 입력 전달
logits

tensor([[ 0.0098, -0.0113,  0.0303,  0.0089,  0.1280,  0.0408,  0.0147,  0.0134,
         -0.0715,  0.0173]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [35]:
pred_probab = nn.Softmax(dim=1)(logits) #Softmax는 모델이 출력한 각 클래스에 대한 점수를 확률 값으로 변환함
                                        #dim은 차원지정으로 각 클래스에 대해 확률을 계산하겠다는 의미다. 
                                        # 이 때 10개의 클래스 중에서 확률이 가장 높은 클래스를 찾기위한 계산이 실행됨
pred_probab

tensor([[0.0991, 0.0970, 0.1011, 0.0990, 0.1115, 0.1022, 0.0996, 0.0994, 0.0913,
         0.0998]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [36]:
y_pred = pred_probab.argmax(1)  #argmax() 는 가 장 큰값을 가진 요소의 인덱스를 반환한다
                                #확률이 가장 높은 클래스의 인덱스(예측된 클래스)를 반환함
y_pred.item()

4

In [37]:
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


## 배치 이미지를 인공신경망에 넣기

In [38]:
input_image = torch.rand(3,28,28) #3개의 batch size(=3개의 샘플) 28* 28 무작위 이미지 생성
print(input_image.size()) 

torch.Size([3, 28, 28])


In [39]:
flatten = nn.Flatten()  # # Flatten 레이어 정의 (이미지를 평탄화하여 1차원 벡터로 변환)
flat_image = flatten(input_image)   #3차원이미지를 1차원으로 평탄화( 평탄화를 하면 3, 784가됨)
print(flat_image.size())

torch.Size([3, 784])


In [40]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20) # 완전 연결층 (28*28 크기의 입력을 20차원의 출력으로 변환)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [41]:
hidden1[1,:]

tensor([-0.0612,  0.4974, -0.2536, -0.3341,  0.2775, -0.2063,  0.2912,  0.3169,
         0.0920,  0.3634,  0.5115,  0.2207, -0.5207,  0.2055,  0.6799,  0.4224,
         0.0033,  0.5624, -0.6482, -0.1190], grad_fn=<SliceBackward0>)

In [42]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2457,  0.6823, -0.3665, -0.3054, -0.2574, -0.1055,  0.4251,  0.2112,
         -0.1701,  0.2264, -0.0447, -0.2398, -0.2520,  0.6438,  0.4460,  0.4927,
          0.1897,  0.2440, -0.6202, -0.1820],
        [-0.0612,  0.4974, -0.2536, -0.3341,  0.2775, -0.2063,  0.2912,  0.3169,
          0.0920,  0.3634,  0.5115,  0.2207, -0.5207,  0.2055,  0.6799,  0.4224,
          0.0033,  0.5624, -0.6482, -0.1190],
        [ 0.2330,  0.5569, -0.4399, -0.3052,  0.1686, -0.2412,  0.4890,  0.3278,
         -0.0456,  0.1872, -0.0890,  0.1861, -0.2933,  0.4187,  0.5743,  0.0200,
          0.6499,  0.0979, -0.4231, -0.1286]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2457, 0.6823, 0.0000, 0.0000, 0.0000, 0.0000, 0.4251, 0.2112, 0.0000,
         0.2264, 0.0000, 0.0000, 0.0000, 0.6438, 0.4460, 0.4927, 0.1897, 0.2440,
         0.0000, 0.0000],
        [0.0000, 0.4974, 0.0000, 0.0000, 0.2775, 0.0000, 0.2912, 0.3169, 0.0920,
         0.3634, 0.5115, 0.2207, 0.0000, 0.2055, 0.67

## 배치 이미지를 인공신경망에 넣기

In [43]:
# 여러 층을 순차적으로 연결한 신경망 모듈을 정의.
# 입력이 첫 번째 층을 지나면 두 번째 층으로, 그리고 그 다음 층으로 계속 전달
seq_modules = nn.Sequential( # 
    flatten, # 입력 텐서를 평탄화
    layer1, # 완전 연결층으로, 평탄화된 이미지 벡터를 입력 받아 20개의 특성을 출력
    nn.ReLU(), #활성화 함수 ReLU를 적용하여 음수 값을 0으로 변환. 이를 통해 비선형성을 도입
    nn.Linear(20, 10)   # 완전 연결층으로, 20개의 특성을 받아 10개의 출력을 생성 
                        # 10개의 클래스에 대한 예측값(logits)을 생성하는 데 사용
)

logits = seq_modules(input_image)
logits

tensor([[-0.3398,  0.0816,  0.1478, -0.0609,  0.0675,  0.0871, -0.0525,  0.2202,
         -0.0064, -0.0505],
        [-0.3494,  0.1336, -0.0115, -0.1035,  0.0405,  0.0984, -0.0946,  0.1502,
         -0.2442,  0.0565],
        [-0.3332, -0.0158,  0.1282,  0.0039,  0.2118,  0.0633, -0.0229,  0.1547,
         -0.0295, -0.1377]], grad_fn=<AddmmBackward0>)

In [44]:
softmax = nn.Softmax(dim=1) #PyTorch에서 제공하는 Softmax 함수를 정의
pred_probab = softmax(logits) #각 클래스에 대한 확률 값을 반환
pred_probab # 각 입력 샘플에 대해 10개의 클래스에 속할 확률 값을 가지는 텐서

tensor([[0.0698, 0.1064, 0.1137, 0.0923, 0.1049, 0.1070, 0.0930, 0.1222, 0.0974,
         0.0932],
        [0.0720, 0.1167, 0.1009, 0.0920, 0.1063, 0.1126, 0.0929, 0.1186, 0.0800,
         0.1080],
        [0.0707, 0.0972, 0.1122, 0.0991, 0.1220, 0.1052, 0.0965, 0.1152, 0.0959,
         0.0860]], grad_fn=<SoftmaxBackward0>)

## 모델 매개변수

In [45]:
print(f"Model structure: {model}\n\n") 

for name, param in model.named_parameters(): #모델의 모든 파라미터에 대한 정보를 가져옴
    print(f"Layer: {name}, | size: {param.size()} | Values: {param[:2]}")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight, | size: torch.Size([512, 784]) | Values: tensor([[-0.0302,  0.0263,  0.0275,  ...,  0.0108, -0.0121, -0.0333],
        [ 0.0172, -0.0078,  0.0148,  ...,  0.0051, -0.0244,  0.0104]],
       device='cuda:0', grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.0.bias, | size: torch.Size([512]) | Values: tensor([0.0060, 0.0277], device='cuda:0', grad_fn=<SliceBackward0>)
Layer: linear_relu_stack.2.weight, | size: torch.Size([512, 512]) | Values: tensor([[-0.0418, -0.0397,  0.0408,  ..., -0.0275, -0.0279, -0.0196],
        [-0.0336, -0.0105,  0.0083,  ...,  0.0016, -0.0087, -0.0146]],
       device='cuda:0', grad_fn=<SliceBac

## 손실함수가 있는 가장 간단한 인공신경망

In [46]:
import torch

x = torch.ones(5) #입력
y = torch.zeros(3) #타깃

w = torch.randn(5, 3, requires_grad = True) #랜덤하게 초기화된 가중치 행렬로, 크기는 (5, 3). 입력 x와 연산할 가중
# requires_grad=True: 이 가중치가 학습 가능한 파라미터임을 명시하여 역전파를 통해 기울기가 계산되도록 함

b = torch.randn(3, requires_grad=True) # 크기가 3인 랜덤한 편향 벡터를 생성
# 애도 기울기가 계산되도록 함

z = torch.matmul(x, w) + b 
#행렬 곱을 수행. 여기서 x는 크기 [5]의 벡터이고 w는 크기 [5, 3]의 가중치 행렬이므로, 이 둘을 곱하면 크기 [3]인 벡터가 됨

loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)
# 시그모이드 함수와 이진 교차 엔트로피 손실을 결합한 손실함수. 
# 로짓 z와 타깃 y를 사용해 손실 값을 계산하며, 이 값은 모델 예측의 정확도를 평가하는 데 사용

In [47]:
w

tensor([[ 2.2219,  0.8247,  0.2360],
        [ 0.9419, -0.5091,  1.0514],
        [-0.5949, -0.2719, -0.2771],
        [ 1.1201, -1.0129,  1.8684],
        [-0.9022, -2.8916, -1.4446]], requires_grad=True)

In [48]:
b

tensor([ 0.6487, -0.3163,  0.1187], requires_grad=True)

In [49]:
z

tensor([ 3.4355, -4.1771,  1.5528], grad_fn=<AddBackward0>)

In [50]:
loss

tensor(1.7424, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

## gradient function

In [51]:
print(f"z와 관련된 그래디언트 함수 = {z.grad_fn}")
print(f"loss와 관련된 그래디언트 함수{loss.grad_fn}")

z와 관련된 그래디언트 함수 = <AddBackward0 object at 0x000001F1AE1D7250>
loss와 관련된 그래디언트 함수<BinaryCrossEntropyWithLogitsBackward0 object at 0x000001F1FDF055E0>


In [52]:
loss.backward() # 호출하면 그래디언트가 계산된다, 한 번만 가능, 
                # 역전파 수행으로 파라미터에 대한 기울기가 계산됨=> 오차를 줄이기위해 가중치를 얼마나 조정해야하는지 나옴
print(w.grad) # 각 파라미터가 손실에 얼마나 영향을 미치는지
print(b.grad) # 각 파라미터가 손실에 얼마나 영향을 미치는지

tensor([[0.3229, 0.0050, 0.2751],
        [0.3229, 0.0050, 0.2751],
        [0.3229, 0.0050, 0.2751],
        [0.3229, 0.0050, 0.2751],
        [0.3229, 0.0050, 0.2751]])
tensor([0.3229, 0.0050, 0.2751])


## 일부 그래디언트 계산이 필요없다면

In [53]:
z = torch.matmul(x, w) + b # 선형변환을 수행
print(z.requires_grad) #텐서가 기울기를 추적하고 있는지 여부 즉, 역전파를 통해 z에 대한 기울기를 계산할 수 있는지

# with에서만 비활성화
with torch.no_grad():# torch.no_grad() 블록 내에서는 기울기 계산이 비활성화(학습과 기울기 계산이 필요 없는 상황에서 사용)
    z = torch.matmul(x, w) + b#
print(z.requires_grad)

True
False


In [54]:
z = torch.matmul(x, w) + b #torch.matmul(x, w): x와 w를 행렬 곱하여, 입력과 가중치의 선형 조합을 계산
z_det = z.detach()  # z.detach()**는 텐서 z에서 기울기 추적을 분리하여 **새로운 텐서 z_det**를 만듦
                    # z와 같은 값을 가지지만, 그래디언트를 추적하지 않는 텐서를 생성

print(z_det.requires_grad) #기울기 추적 여부

False
